In [4]:
## Load context
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re
import os
import pandas as pd
import numpy as np
hostname = os.uname()[1]

from pyspark.sql import SparkSession

# BEN'S MASTER: 192.168.2.87
# OUR MASTER:   192.168.2.203

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.203:7077") \
        .appName(f"load_local_comments; hostname: {hostname}")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.blockManager.port", "10025")\
        .config("spark.driver.blockManager.port", "10026")\
        .config("spark.driver.port", "10027")\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.executor.cores",6)\
        .getOrCreate()
        
#spark_session = SparkSession\
#        .builder\
#        .master("spark://192.168.2.203:7077") \
#        .appName(f"load_local_comments; hostname: {hostname}")\
#        .config("spark.dynamicAllocation.enabled", True)\
#        .config("spark.shuffle.service.enabled", True)\
#        .config("spark.executor.cores",6)\
#        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext
spark_context.uiWebUrl

SyntaxError: invalid syntax (<ipython-input-4-47a672d8b394>, line 28)

In [1]:
from pyspark import SparkConf, SparkContext  
## Load context
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re
import os
import pandas as pd
import numpy as np
hostname = os.uname()[1]

from pyspark.sql import SparkSession
conf = (SparkConf()    
   .setMaster("spark://192.168.2.203:7077")  
   .setAppName("mytest")    
   .set("spark.executor.cores",1)  
   .set("spark.dynamicAllocation.enabled", False)  
   .set("spark.shuffle.service.enabled", False))  
#spark_context = SparkContext(conf = conf)  
spark_session = SparkSession.builder.config(conf=conf).getOrCreate()


In [2]:
import pyspark.sql.functions as f

df_pd = pd.read_json("sample_reddit_comments.json", lines=True)
df = spark_session.createDataFrame(df_pd).cache()

In [ ]:
from pyspark.sql import functions as F

udf_strip_lower = F.udf(lambda comment: (re.sub(r'\W+', ' ', comment).lower().strip()), 'string')
df_1 = df.select('body').withColumn('body', udf_strip_lower('body'))

#df.select('body').withColumn('body', f.lower(f.col('body'))).take(5)

In [ ]:
df_2 = df_1.withColumn('word', f.explode(f.split(f.col('body'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

df_2.show()